In [5]:
import os
import pandas as pd

def load_predictions_data():
    predictions_path=os.path.join("datasets", "predictions_prod_smallest.csv")
    return pd.read_csv(predictions_path, dtype={"predictions_route_id": str, "predictions_stop_id": str})

In [6]:
data = load_predictions_data()

In [78]:
data.head()

,predictions_id,predictions_trip_id,predictions_arrival_time,predictions_boarding_status,predictions_departure_time,predictions_stop_id,predictions_stop_sequence,predictions_stops_away,predictions_vehicle_event_id,predictions_file_timestamp,predictions_route_id,predictions_vehicle_id,predictions_direction_id
0,648187443,39783376,1.553863e+09,NaN,1.553863e+09,70040,10,1.0,10212508,1553862833,Blue,B-545C365A,1
1,648187455,39988585-20:30-FKenmoreStMaryC,1.553863e+09,NaN,1.553863e+09,70203,620,1.0,10212521,1553862833,Green-C,G-10142,1
2,648187514,40033949,1.553864e+09,NaN,1.553864e+09,70007,30,8.0,10214673,1553862833,Orange,O-545C36A0,1
3,648187604,ADDED-1553782585,1.553864e+09,NaN,1.553864e+09,70156,580,9.0,10214713,1553862833,Green-D,G-10152,1
4,648187732,40033948,1.553863e+09,NaN,1.553863e+09,70005,20,2.0,10213047,1553862833,Orange,O-545C364E,1


In [7]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(data, data["predictions_route_id"]):
    strat_train_set = data.loc[train_index]
    strat_test_set = data.loc[test_index]

In [82]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from datetime import datetime
import numpy as np

class DropColumnsTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, include_stops_away=False, include_route_id=False):
        self.include_stops_away = include_stops_away
        self.include_route_id = include_route_id
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X.drop(["predictions_id", 
                       "predictions_trip_id", 
                       "predictions_stop_sequence",
                       "predictions_vehicle_event_id",
                       "predictions_vehicle_id",
                       "predictions_file_timestamp",],
                      axis=1)

class IsStoppedTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        result = X.copy()
        result['is_stopped'] = X['predictions_boarding_status'].notnull()
        return result.drop('predictions_boarding_status', axis=1)

class TimestampTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        new_data = X.copy()
        datetimes = new_data['predictions_file_timestamp'].apply(
            lambda x: datetime.fromtimestamp(x),
        )
        predictions_day_of_week = datetimes.apply(lambda x: x.weekday())
        predictions_time_bin = datetimes.apply(self._timestamp_bin)
        new_data['predictions_day_of_week'] = predictions_day_of_week
        new_data['predictions_time_bin'] = predictions_time_bin
        return new_data
    
    def _timestamp_bin(self, timestamp):
        return timestamp.hour * 4 + timestamp.minute // 15

class ColumnOneHotEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        enc = OneHotEncoder()
        one_hotted = enc.fit_transform(X[self.column].values.reshape(-1,1))
        return np.concatenate((X.drop(self.column, axis=1), one_hotted), axis=1)
    
class DebugTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        print(X.info())
        return X

In [91]:
from sklearn.pipeline import Pipeline

predictions_pipeline = Pipeline([
    ('timestamp', TimestampTransformer()),
    ('is_stopped', IsStoppedTransformer()),
    ('drop', DropColumnsTransformer()),
    ('debug', DebugTransformer()),
    ('1hot', OneHotEncoder(categorical_features=[2, 4, 6, 7]))
])


In [92]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder

noncat_list = ['predictions_id',
               'predictions_trip_id',
               'predictions_arrival_time',
               'predictions_boarding_status',
               'predictions_departure_time',
               'predictions_stop_sequence',
               'predictions_stops_away',
               'predictions_vehicle_event_id',
               'predictions_file_timestamp',
               'predictions_vehicle_id',
               'predictions_direction_id']

full_pipeline = ColumnTransformer([
    ("overall", predictions_pipeline, noncat_list),
    ("1hot", OneHotEncoder(), [2, 4, 6, 7])
    #("cat", OneHotEncoder(), [10, 20, 101])
])

fitted = predictions_pipeline.fit_transform(strat_train_set.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 213414 to 6049928
Data columns (total 9 columns):
predictions_arrival_time      4 non-null float64
predictions_departure_time    5 non-null float64
predictions_stop_id           5 non-null object
predictions_stops_away        5 non-null float64
predictions_route_id          5 non-null object
predictions_direction_id      5 non-null int64
predictions_day_of_week       5 non-null int64
predictions_time_bin          5 non-null int64
is_stopped                    5 non-null bool
dtypes: bool(1), float64(3), int64(3), object(2)
memory usage: 365.0+ bytes
None


/Users/gdurazo/mbta/ml_exploration/env/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


ValueError: could not convert string to float: 'Red'

In [90]:
fitted.head()

,predictions_arrival_time,predictions_departure_time,predictions_stop_id,predictions_stops_away,predictions_route_id,predictions_direction_id,predictions_day_of_week,predictions_time_bin,is_stopped
213414,1.553748e+09,1.553748e+09,70079,4.0,Red,0,2,94,False
2301512,1.553614e+09,1.553614e+09,70087,0.0,Red,0,1,42,False
6484008,NaN,1.553263e+09,70061,2.0,Red,0,4,35,False
6973123,1.553343e+09,1.553343e+09,70200,2.0,Green-E,1,5,28,False
6049928,1.553511e+09,1.553511e+09,70021,12.0,Orange,1,0,21,False


In [28]:
full_pipeline.fit_transform(strat_train_set)

array([[1553748268.0, 1553748286.0, 4.0, ..., 2, 94, False],
       [1553614221.0, 1553614292.0, 0.0, ..., 1, 42, False],
       [nan, 1553263317.0, 2.0, ..., 4, 35, False],
       ...,
       [1553549167.0, 1553549268.0, 6.0, ..., 0, 64, False],
       [1553777861.0, 1553777921.0, 1.0, ..., 3, 31, False],
       [1553782344.0, 1553782393.0, 16.0, ..., 3, 34, False]],
      dtype=object)

In [45]:
enc = ColumnOneHotEncoder("predictions_id")
fitted = enc.fit_transform(strat_train_set)
fitted

/Users/gdurazo/mbta/ml_exploration/env/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


TypeError: 'CClass' object is not callable

In [65]:
data.values[:, 0]

array([648187443, 648187455, 648187514, ..., 648184248, 648184356,
       648184365], dtype=object)

In [67]:
np.array([1, 2, 3])

array([1, 2, 3])

In [69]:
np.array([[1, 2, 3], [2, 3, 4]])

array([[1, 2, 3],
       [2, 3, 4]])

In [72]:
a = np.array([[1, 2, 3], [2, 3, 4]])
b = np.array([[5, 6, 7], [8, 9, 0]])

c = np.array([1,2,3])
d = np.array([4,5,6])

np.concatenate((a, b), axis=1)

array([[1, 2, 3, 5, 6, 7],
       [2, 3, 4, 8, 9, 0]])